# Working with uDALES Field Data

This tutorial describes how to read and process field data output of the LES code uDALES using Python. It covers important concepts such as:
- Grid layout and variable locations
- Averaging procedures used in uDALES output
- Loading different types of field data

The **UDBase** post-processing class contains methods to load field data:
- **load_stat_xyt**: Time- and slab-averaged statistics (xytdump.expnr.nc)
- **load_stat_t**: Time-averaged statistics (tdump.expnr.nc)
- **load_field**: Instantaneous 3D data (fielddump.expnr.nc)
- **load_slice**: Instantaneous 2D slices (Xslicedump.expnr.nc)

## 1. Import Libraries and Setup

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Add the uDALES python path
sys.path.insert(0, '../tools/python')

from udbase import UDBase

# Create an instance of the UDBase class
expnr = '110'
expdir = '../experiments/110'
sim = UDBase(case_dir=expdir)

## 2. uDALES Grid Layout

uDALES uses a **staggered grid** - not all variables are defined at the same location. This is computationally advantageous but requires care with plotting.

### Grid Staggering (x-z view)

```
                      w(i,j,k+1)
                          ^
zm(k+1) --        --------|--------
                  |               |
                  |   c(i,j,k)    |
  zt(k) --  u(i,j,k) --> o      ---> u(i+1,j,k)
                  |               |
                  |       ^       |
  zm(k) --        --------|--------
                      w(i,j,k)
                  |       |       |
                xm(i)   xt(i)   xm(i+1)
```

### Coordinate Definitions

- **xm(i)** = (i-1) × dx;  **xt(i)** = (i-1/2) × dx
- **ym(j)** = (j-1) × dy;  **yt(j)** = (j-1/2) × dy
- **zm(k)** = (k-1) × dz;  **zt(k)** = (k-1/2) × dz

### Grid Increments

- **dzt(k)** = zm(k+1) - zm(k) : cell height
- **dzm(k)** = zt(k) - zt(k-1) : distance between cell centers

In [ ]:
# Display grid dimensions and domain size
print(f"Grid dimensions: {sim.itot} x {sim.jtot} x {sim.ktot}")
print(f"Domain size: {sim.xlen} x {sim.ylen} x {sim.zsize} m")

## 3. load_stat_xyt: Time- and Slab-Averaged Data

In [ ]:
# View available variables in xytdump file
sim.load_stat_xyt()

In [ ]:
help(sim.load_stat_xyt)

In [ ]:
# Load time- and slab-averaged streamwise velocity
uxyt = sim.load_stat_xyt('uxyt')

# Plot mean velocity profile
plt.figure(figsize=(8, 6))
plt.plot(uxyt, sim.zm, linewidth=2)
plt.ylabel('z [m]')
plt.xlabel('⟨ū⟩ [m/s]')
plt.title('Mean Streamwise Velocity Profile')
plt.grid(True, alpha=0.3)
plt.tight_layout()

## 4. Momentum Flux Analysis

uDALES separates momentum fluxes into turbulent and dispersive components:
- **Turbulent flux**: ⟨u'w'⟩ (from upwpxyt)
- **Dispersive flux**: ⟨u"w"⟩ (from uwxyt)
- **Total flux**: ⟨u'w'⟩ + ⟨u"w"⟩

In [ ]:
# Load turbulent and dispersive momentum flux
upwpxyt = sim.load_stat_xyt('upwpxyt')  # Turbulent flux
uwxyt = sim.load_stat_xyt('uwxyt')      # Dispersive flux
total_flux = upwpxyt + uwxyt

# Plot comparison
plt.figure(figsize=(8, 6))
plt.plot(upwpxyt, sim.zt, label='Turbulent')
plt.plot(uwxyt, sim.zt, label='Dispersive')
plt.plot(total_flux, sim.zt, label='Total', linewidth=2)
plt.ylabel('z [m]')
plt.xlabel('⟨uw⟩ [m²/s²]')
plt.legend()
plt.grid(True, alpha=0.3)

## 5. load_stat_t: Time-Averaged 3D Data

In [ ]:
help(sim.load_stat_t)

In [ ]:
# Load time-averaged temperature
T_avg = sim.load_stat_t('T')

# Extract horizontal slice at z=20m
z_idx = np.argmin(np.abs(sim.zt - 20))
T_slice = T_avg[z_idx, :, :]

# Plot
plt.figure(figsize=(8, 6))
plt.contourf(sim.xt, sim.yt, T_slice, levels=20)
plt.colorbar(label='Temperature [K]')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.title('Time-averaged Temperature at z=20m')

## 6. load_field: Instantaneous 3D Data

In [ ]:
help(sim.load_field)

In [ ]:
# Load velocity at t=3600s
u = sim.load_field('u', time=3600)

# Horizontal slice at z=10m
z_idx = np.argmin(np.abs(sim.zt - 10))
u_slice = u[z_idx, :, :]

# Vertical profile (domain average)
u_profile = u.mean(dim=['y', 'x'])

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].contourf(sim.xt, sim.yt, u_slice, levels=20)
axes[0].set_xlabel('x [m]')
axes[0].set_ylabel('y [m]')
axes[0].set_title(f'u at z={sim.zt[z_idx]:.1f}m')

axes[1].plot(u_profile, sim.zt)
axes[1].set_xlabel('⟨u⟩ [m/s]')
axes[1].set_ylabel('z [m]')
axes[1].set_title('Domain-averaged u profile')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()

## 7. load_slice: 2D Slice Data

In [ ]:
help(sim.load_slice)

In [ ]:
# Load XZ slice at y=constant
u_xz = sim.load_slice('xz', 'u', time=3600)

# Plot
plt.figure(figsize=(10, 6))
plt.contourf(sim.xt, sim.zt, u_xz.T, levels=20)
plt.colorbar(label='u [m/s]')
plt.xlabel('x [m]')
plt.ylabel('z [m]')
plt.title('Streamwise velocity (XZ slice)')
plt.tight_layout()

## 8. Averaging Procedures in uDALES

uDALES uses several types of averaging to analyze turbulent urban flows:

### 1. Slab Averaging (⟨·⟩)
Average in x and y directions, produces vertical profiles:
- Used in `load_stat_xyt`
- Example: ⟨u⟩(z, t)

### 2. Time Averaging (‾)
Average over time interval, reduces statistical noise:
- Used in all stat outputs
- Example: ū(x, y, z)

### 3. Reynolds Decomposition
Separates instantaneous field into mean and fluctuating components:
- φ = φ̄ + φ'
- φ̄: mean component
- φ': fluctuating component

### 4. Dispersive Decomposition
Separates slab-averaged mean into uniform and spatially-varying parts:
- φ̄ = ⟨φ̄⟩ + φ̄"
- ⟨φ̄⟩: slab-averaged mean
- φ̄": spatial deviation from slab average

### Key Relationships
- **Turbulent flux**: ⟨u'w'⟩ (from upwpxyt)
- **Dispersive flux**: ⟨ū"w̄"⟩ (from uwxyt)
- **Total flux**: ⟨u'w'⟩ + ⟨ū"w̄"⟩

## 9. Summary

### Key Takeaways
- ✅ UDBase provides unified interface for all field data types
- ✅ Grid is staggered - pay attention to variable locations (u, v, w at edges; scalars at centers)
- ✅ xarray DataArrays provide labeled dimensions for easy manipulation
- ✅ Methods handle file I/O and coordinate management automatically

### Data Loading Methods
| Method | File | Description | Dimensions |
|--------|------|-------------|------------|
| `load_stat_xyt` | xytdump | Time & slab averaged | (z, time) |
| `load_stat_t` | tdump | Time averaged 3D | (z, y, x) |
| `load_field` | fielddump | Instantaneous 3D | (z, y, x) or (time, z, y, x) |
| `load_slice` | *slicedump | Instantaneous 2D | (dim1, dim2) |

### Next Steps
- **facets_tutorial.ipynb** - Surface data analysis and energy balance
- **geometry_tutorial.ipynb** - Creating and manipulating urban geometries
- `tools/python/fields_example.py` - Complete working examples
- `tools/python/QUICK_REFERENCE.py` - API reference guide